<a href="https://colab.research.google.com/github/bhushanmandava/Google_Stock_Prediction_RNN-LSTM_Scratch-/blob/main/ElmanRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
class elmanRNNUnit(nn.Module):
  def __init__(self,embedding_dim):
    super().__init__()
    self.embedding_dim =embedding_dim
    self.wh= nn.Parameter(torch.randn(self.embedding_dim,self.embedding_dim))
    self.uh= nn.Parameter(torch.randn(self.embedding_dim,self.embedding_dim))#creating random weights for our input values adn hidden values
    self.bh =nn.Parameter(torch.zeros(self.embedding_dim))#creating zero vec for biases
  def forward(self,x,h):
    return torch.tanh(x@self.wh+h@self.uh+self.bh)


In [3]:
class elmanRNN(nn.Module):
  def __init__(self,embedding_dim,n_layers):
    super().__init__()
    self.embedding_dim=embedding_dim
    self.n_layers=n_layers
    self.rnn_units = nn.ModuleList(
        [elmanRNNUnit(embedding_dim) for _ in range(n_layers)]
    )
  def forward(self,x):
    batch_size,seq_len,emb_dim=x.shape
    h_perv =[
        torch.zeros(batch_size,emb_dim)
        for _ in range(self.n_layers)
    ]
    outp=[]
    for t in range(seq_len):
      x_t =x[:,t]
      for l,r_unit in enumerate(self.rnn_units):
        h_new= r_unit(x_t,h_perv[l])
        h_perv[l]=h_new
        x_t=h_new
      outp.append(h_new)
    return torch.stack(outp,dim=1)


In [5]:
embedding_dim = 10
sequence_length = 5
batch_size = 16
num_classes = 2
n_layers = 2
epochs = 50


X_train = torch.randn(batch_size, sequence_length, embedding_dim)
y_train = (X_train.sum(dim=(1, 2)) > 0).long()

model = elmanRNN(embedding_dim, n_layers)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)[:, -1, :]  # Take the last time step output
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}")


Epoch [0/50], Loss: 2.6531
Epoch [10/50], Loss: 2.1329
Epoch [20/50], Loss: 2.0422
Epoch [30/50], Loss: 1.7944
Epoch [40/50], Loss: 1.7227
